# use Onnx to load converted model

### onnx

In [1]:
import onnx

# Load the ONNX model
model = onnx.load("onnx/yolo2.onnx")

# Check that the IR is well formed
onnx.checker.check_model(model)

# Print a human readable representation of the graph
print(onnx.helper.printable_graph(model.graph))

graph torch-jit-export (
  %1[FLOAT, 1x3x416x416]
) initializers (
  %2[FLOAT, 32x3x3x3]
  %3[FLOAT, 32]
  %4[FLOAT, 32]
  %5[FLOAT, 32]
  %6[FLOAT, 32]
  %7[FLOAT, 64x32x3x3]
  %8[FLOAT, 64]
  %9[FLOAT, 64]
  %10[FLOAT, 64]
  %11[FLOAT, 64]
  %12[FLOAT, 128x64x3x3]
  %13[FLOAT, 128]
  %14[FLOAT, 128]
  %15[FLOAT, 128]
  %16[FLOAT, 128]
  %17[FLOAT, 64x128x1x1]
  %18[FLOAT, 64]
  %19[FLOAT, 64]
  %20[FLOAT, 64]
  %21[FLOAT, 64]
  %22[FLOAT, 128x64x3x3]
  %23[FLOAT, 128]
  %24[FLOAT, 128]
  %25[FLOAT, 128]
  %26[FLOAT, 128]
  %27[FLOAT, 256x128x3x3]
  %28[FLOAT, 256]
  %29[FLOAT, 256]
  %30[FLOAT, 256]
  %31[FLOAT, 256]
  %32[FLOAT, 128x256x1x1]
  %33[FLOAT, 128]
  %34[FLOAT, 128]
  %35[FLOAT, 128]
  %36[FLOAT, 128]
  %37[FLOAT, 256x128x3x3]
  %38[FLOAT, 256]
  %39[FLOAT, 256]
  %40[FLOAT, 256]
  %41[FLOAT, 256]
  %42[FLOAT, 512x256x3x3]
  %43[FLOAT, 512]
  %44[FLOAT, 512]
  %45[FLOAT, 512]
  %46[FLOAT, 512]
  %47[FLOAT, 256x512x1x1]
  %48[FLOAT, 256]
  %49[FLOAT, 256]
  %50[FLOAT, 256]

### prepare image

In [2]:
from PIL import Image 
import numpy as np

In [3]:
imgfile = './data/dog.jpg'
img = Image.open(imgfile).resize( (416, 416) )
img_arr = np.array(img)
img_arr = np.expand_dims(img_arr, -1)
img_arr = np.transpose(img_arr, (3,2,0,1))

In [4]:
img_arr.shape

(1, 3, 416, 416)

### prepare model

https://ptorch.com/news/95.html

In [5]:
%%time
# import onnx_caffe2.backend as backend_1
import onnx_tf.backend as backend
import numpy as np

rep = backend.prepare(model, device="CUDA:0") # or "CPU"

/usr/local/lib/python2.7/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
/usr/local/lib/python2.7/dist-packages/onnx_tf/backend.py:677: UserWarning: Unsupported kernel_shape attribute by Tensorflow in Conv operator. The attribute will be ignored.
  UserWarning)


CPU times: user 37.8 s, sys: 855 ms, total: 38.7 s
Wall time: 38.4 s


### inference

In [6]:
%%time
# For the Caffe2 backend:
#     rep.predict_net is the Caffe2 protobuf for the network
#     rep.workspace is the Caffe2 workspace for the network
#       (see the class onnx_caffe2.backend.Workspace)
outputs = rep.run(img_arr.astype(np.float32))
# To run networks with more than one input, pass a tuple
# rather than a single numpy ndarray.
print(outputs[0].shape)

(1, 425, 13, 13)
CPU times: user 8.19 s, sys: 1 s, total: 9.19 s
Wall time: 3.08 s
